# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # YOUR CODE HERE
    # NHL data modifications and cleaning.
    nhl_df_2018 = nhl_df.where(nhl_df['year'] == 2018).drop(['GP', 'year', 'League', 'PTS', 'PTS%', 'GF', 
                                                             'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'OL'], axis=1).dropna()
    nhl_df_2018.replace({'Atlantic Division':np.nan, 'Metropolitan Division':np.nan, 'Central Division':np.nan,
                        'Pacific Division':np.nan}, inplace=True)
    nhl_df_2018['team'].replace({'New Jersey Devils*': 'New York', 'Carolina Hurricanes': 'Raleigh',
                                'Colorado Avalanche*':'Denver', 'Vegas Golden Knights*': 'Las Vegas',
                                'Anaheim Ducks*':'Los Angeles', 'Arizona Coyotes': 'Phoenix'}, inplace=True)
    nhl_df_2018 = nhl_df_2018.dropna().reset_index().drop('index',axis=1)
    
    # Metropolitan Regions Modification and cleaning.
    nhl_population = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    nhl_population.replace('—', np.nan, inplace=True)
    nhl_population.replace(r'^\[\w+\s\d+\]$', np.nan, regex=True,inplace=True)
    nhl_population.replace({'New York City': 'New York', 'San Francisco Bay Area': 'San Jose', 'Dallas–Fort Worth': 'Dallas',
                           'Washington, D.C.': 'Washington', 'Minneapolis–Saint Paul':"Minnesota", 
                            'Miami–Fort Lauderdale': 'Florida', 'Tampa Bay Area': 'Tampa Bay'}, inplace=True)
    nhl_population = nhl_population.dropna().reset_index().drop('index', axis=1)
    
    # added population and region in nhl_df_2018 dataframe.
    region= {}
    population= {}
    
    for area_idx, area in enumerate(nhl_population['Metropolitan area'].to_list(), start=0):
        for team, team_idx in zip(nhl_df_2018['team'].to_list(), nhl_df_2018.index.to_list()):
            if re.match(area, team) != None:
                region[team_idx] = area
                population[team_idx] = nhl_population['Population (2016 est.)[8]'][area_idx]

    region_df = pd.Series(region, name='region')

    population_df = pd.Series(population, name='population')
    population_df = pd.to_numeric(population_df)
    
    nhl_df_2018['region'] = region_df
    nhl_df_2018['population'] = population_df
    nhl_df_2018['team'].replace({'New York':'New Jersey Devils*', 'Raleigh':'Carolina Hurricanes',
                                'Denver':'Colorado Avalanche*', 'Las Vegas': 'Vegas Golden Knights*',
                                'Los Angeles':'Anaheim Ducks*', 'Phoenix':'Arizona Coyotes'}, inplace=True)
    
    # change data type from W and L columns from String to Int.
    nhl_df_2018['W'] = nhl_df_2018['W'].astype(float)
    nhl_df_2018['L'] = nhl_df_2018['L'].astype(float)
    nhl_df_2018 = nhl_df_2018.set_index('region')
    
    # treat newyork and losangeles as a special column because it is have multiple index.
    new_york = nhl_df_2018.loc['New York']
    los_angeles = nhl_df_2018.loc['Los Angeles']
    
    ny_wlratio = nhl_df_2018.loc['New York'][['W','L']].sum() / len(nhl_df_2018.loc['New York'][['W','L']])
    la_wlratio = nhl_df_2018.loc['Los Angeles'][['W','L']].sum() / len(nhl_df_2018.loc['Los Angeles'][['W','L']])

    # dropping the exist newyork and losangeles entry and replace it to the new one.
    nhl_df_2018.drop(['New York', 'Los Angeles'], inplace=True)
    
    # drop the team column, it is not use.
    nhl_df_2018.drop('team', axis=1, inplace=True)
    
    # compute los angeles and newyork entries to make it just one entry.
    nhl_df_2018.loc['New York'] = [ny_wlratio['W'], ny_wlratio['L'], new_york['population'].to_list()[0]]
    nhl_df_2018.loc['Los Angeles'] = [la_wlratio['W'], la_wlratio['L'], los_angeles['population'].to_list()[0]]
    
    # create wl_ratio column in nhl dataframe.
    nhl_df_2018['wl_ratio'] = nhl_df_2018['W'] / (nhl_df_2018['W']+nhl_df_2018['L'])
    
    # get all entry from nhl dataframe adding it into the list. 
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    for population, ratio in zip(pd.to_numeric(nhl_df_2018['population']), pd.to_numeric(nhl_df_2018['wl_ratio'])):
        population_by_region.append(population)
        win_loss_by_region.append(ratio)
        
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(nhl_correlation())

0.012308996455744244


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    # NBA data modifications and cleaning.
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df.drop(['W/L%', 'PS/G', 'PA/G', 'SRS' ,'League', 'GB'], axis=1, inplace=True)
    nba_df = nba_df.where(nba_df['year'] == 2018).dropna()
    nba_df.drop('year', axis=1, inplace=True)
    
    # cities modifications and cleaning.
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.drop(['NFL','MLB','NHL'], axis=1, inplace=True)
    cities.replace([r'^\[\w+\s\d+\]$', '—'], np.nan, regex=True,inplace=True)
    cities = cities.dropna().reset_index()
    cities.rename({'Population (2016 est.)[8]':'population'}, axis=1, inplace=True)
    cities.drop('index', axis=1, inplace=True)
    cities.replace({'New York City': 'New York', 'San Francisco Bay Area': 'Golden State', 'Dallas–Fort Worth': 'Dallas',
                   'Washington, D.C.': 'Washington', 'Minneapolis–Saint Paul':'Minnesota', 'Miami–Fort Lauderdale': 'Miami',
                   'Salt Lake City': 'Utah' ,'Indianapolis': 'Indiana'}, inplace=True) 

    # create region and population columns
    population = {}
    region = {}
    for idx, city in enumerate(cities['Metropolitan area'].to_list(), start=0):
        for city2, idx2 in zip(nba_df['team'].to_list(), nba_df.index.to_list()):
            if re.match(city, city2) != None:
                population[idx2] = cities['population'][idx]
                region[idx2] = cities['Metropolitan area'][idx]
            if city2 == 'Brooklyn Nets\xa0(12)':
                population[idx2] = cities['population'][0]
                region[idx2] = cities['Metropolitan area'][0]
        
    nba_df['region'] = pd.Series(region)
    nba_df['population'] = pd.Series(population)
    
    # treat los angeles and new york entries differently.
    nba_df[['W', 'L']] = nba_df[['W', 'L']].astype(float)
    
    la_reg, la_pop = nba_df.iloc[24:26]['region'].to_list()[0], nba_df.iloc[24:26]['population'].to_list()[0]
    la_w, la_l = nba_df.iloc[24:26]['W'].sum() / len(nba_df.iloc[24:26]), nba_df.iloc[24:26]['L'].sum() / len(nba_df.iloc[24:26])
    
    ny_reg, ny_pop = nba_df.iloc[10:12]['region'].to_list()[0], nba_df.iloc[10:12]['population'].to_list()[0]
    ny_w, ny_l = nba_df.iloc[10:12]['W'].sum() / len(nba_df.iloc[10:12]), nba_df.iloc[10:12]['L'].sum() / len(nba_df.iloc[10:12])

    nba_df.drop('team', axis=1, inplace=True)
    nba_df.drop([10, 11, 24, 25], axis=0 , inplace=True)
    nba_df = nba_df.reset_index()
    nba_df.drop('index', axis=1, inplace=True)
    
    nba_df.loc[len(nba_df)] = [ny_w, ny_l, ny_reg, ny_pop]
    nba_df.loc[len(nba_df)] = [la_w, la_l, la_reg, la_pop]
    
    nba_df['wl_ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    nba_df['population'] = nba_df['population'].astype(float)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    for pop, ratio in zip(nba_df['population'], nba_df['wl_ratio']):
        population_by_region.append(pop)
        win_loss_by_region.append(ratio)
        
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(nba_correlation())

-0.17657160252844614


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    
    # mlb data cleaning and modifications.
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df.where(mlb_df['year'] == 2018).dropna()
    mlb_df.drop(['W-L%' ,'League', 'GB', 'year'], axis=1, inplace=True)
    
    # cities modificaions and cleaning.
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.drop(['NFL','NBA','NHL'], axis=1, inplace=True)
    cities.replace([r'^\[\w+\s\d+\]$', '—'], np.nan, regex=True,inplace=True)
    cities = cities.dropna().reset_index()
    cities.rename({'Population (2016 est.)[8]':'population'}, axis=1, inplace=True)
    cities.drop('index', axis=1, inplace=True)
    cities.replace({'San Francisco Bay Area':'San Francisco', 'Dallas–Fort Worth':'Texas', 'Washington, D.C.':'Washington',
                   'Minneapolis–Saint Paul':'Minnesota', 'Miami–Fort Lauderdale':'Miami', 'Tampa Bay Area': 'Tampa Bay',
                   'New York City': 'New York', 'Phoenix':'Arizona', 'Denver':'Colorado'}, inplace=True) 
    
    # create region and population columns
    population = {}
    region = {}
    for idx, city in enumerate(cities['Metropolitan area'].to_list(), start=0):
        for city2, idx2 in zip(mlb_df['team'].to_list(), mlb_df.index.to_list()):
            if re.match(city, city2) != None:
                population[idx2] = cities['population'][idx]
                region[idx2] = cities['Metropolitan area'][idx]
            if city2 == 'Oakland Athletics':
                population[idx2] = cities['population'][2]
                region[idx2] = cities['Metropolitan area'][2]
                
    mlb_df['region'] = pd.Series(region)
    mlb_df['population'] = pd.to_numeric(pd.Series(population))
    
    la = mlb_df.where(mlb_df['region'] == 'Los Angeles').dropna()
    chicago = mlb_df.where(mlb_df['region'] == 'Chicago').dropna()
    ny = mlb_df.where(mlb_df['region'] == 'New York').dropna()
    sf = mlb_df.where(mlb_df['region'] == 'San Francisco').dropna()
    
    la_w, la_l = la['W'].sum() / len(la), la['L'].sum() / len(la)
    chi_w, chi_l = chicago['W'].sum() / len(chicago) , chicago['L'].sum() / len(chicago)
    ny_w, ny_l = ny['W'].sum() / len(ny), ny['L'].sum() / len(ny)
    sf_w, sf_l = sf['W'].sum() / len(sf), sf['L'].sum()/ len(sf)
    
    mlb_df.replace(['New York', 'Chicago', 'Los Angeles', 'San Francisco'], np.nan, inplace=True)
    mlb_df = mlb_df.dropna().reset_index().drop(['team','index'], axis=1)
    
    mlb_df.loc[len(mlb_df)] = [la_w, la_l, 'Los Angeles', la.population.to_list()[0]]
    mlb_df.loc[len(mlb_df)] = [chi_w, chi_l, 'Chicago', chicago.population.to_list()[0]]
    mlb_df.loc[len(mlb_df)] = [ny_w, ny_l, 'New York', ny.population.to_list()[0]]
    mlb_df.loc[len(mlb_df)] = [sf_w, sf_l, 'San Francisco', sf.population.to_list()[0]]
    
    mlb_df['wl_ratio'] = mlb_df['W'] / (mlb_df['W']+mlb_df['L'])

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    for pop, ratio in zip(mlb_df['population'], mlb_df['wl_ratio']):
        population_by_region.append(pop)
        win_loss_by_region.append(ratio)
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(mlb_correlation())

0.1505230448710485


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfl_df = nfl_df.where(nfl_df['year'] == 2018).dropna()
    nfl_df.drop(['DSRS', 'League', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS',
                'T', 'W-L%','year'], axis=1, inplace=True)
    nfl_df.replace(['AFC East', 'AFC North', 'AFC South', 'AFC West' ,'NFC East', 'NFC North', 'NFC South', 'NFC West'], 
                   np.nan, inplace=True)
    nfl_df = nfl_df.dropna()
    nfl_df = nfl_df[['team', 'W', 'L']]
    
    cities.drop(['MLB','NBA','NHL'], axis=1, inplace=True)
    cities.replace([r'^\[\w+\s\d+\]$', '—'], np.nan, regex=True,inplace=True)
    cities = cities.dropna().reset_index()
    cities.rename({'Population (2016 est.)[8]':'population'}, axis=1, inplace=True)
    cities.drop('index', axis=1, inplace=True)
    cities.replace({'New York City': 'New York', 'San Francisco Bay Area': 'San Francisco', 'Dallas–Fort Worth': 'Dallas',
                   'Washington, D.C.': 'Washington', 'Minneapolis–Saint Paul': 'Minnesota', 'Miami–Fort Lauderdale': 'Miami',
                   'Phoenix': 'Arizona', 'Tampa Bay Area': 'Tampa Bay', 'Charlotte': 'Carolina', "Nashville":'Tennessee',
                   'Boston': 'New England'}, inplace=True)
        
    population = {}
    region = {}
    
    for idx, city in enumerate(cities['Metropolitan area'].to_list(), start=0):
        for city2, idx2 in zip(nfl_df['team'].to_list(), nfl_df.index.to_list()):
            if re.match(city, city2) != None:
                population[idx2] = cities['population'][idx]
                region[idx2] = cities['Metropolitan area'][idx]
            if city2 == 'Oakland Raiders':
                population[idx2] = cities['population'][2]
                region[idx2] = cities['Metropolitan area'][2]
    
    nfl_df['population'] = pd.Series(population)
    nfl_df['region'] = pd.Series(region)
    nfl_df = nfl_df.reset_index()
    nfl_df[['W','L','population']] = nfl_df[['W','L','population']].astype(float)
    nfl_df = nfl_df.drop('index', axis=1)

    ny = nfl_df.where(nfl_df['region'] == 'New York').dropna()
    la = nfl_df.where(nfl_df['region'] == 'Los Angeles').dropna()
    sf = nfl_df.where(nfl_df['region'] == 'San Francisco').dropna()
    
    ny_w, ny_l = ny['W'].sum() / len(ny['W']), ny['L'].sum()/len(ny['L'])
    la_w, la_l = la['W'].sum() / len(ny['W']), la['L'].sum()/len(la['L'])
    sf_w, sf_l = sf['W'].sum() / len(sf['W']), sf['L'].sum()/len(sf['L'])
    
    nfl_df.replace(['New York', 'Los Angeles', 'San Francisco'], np.nan, inplace=True)
    nfl_df = nfl_df.dropna()
    nfl_df.reset_index(inplace=True, drop=True)
    
    nfl_df.loc[len(nfl_df)] = ['unkown', ny_w, ny_l, ny['population'].to_list()[0], 'New York',]
    nfl_df.loc[len(nfl_df)] = ['unkown', la_w, la_l, la['population'].to_list()[0], 'Los Angeles']
    nfl_df.loc[len(nfl_df)] = ['unkown', sf_w, sf_l, sf['population'].to_list()[0] ,'San Francisco']
    
    nfl_df = nfl_df.drop('team', axis=1)

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    nfl_df['wl_ratio'] = nfl_df['W'] / (nfl_df['W']+nfl_df['L'])
    
    for ratio, pop in zip(nfl_df['wl_ratio'].to_list(), nfl_df['population'].to_list()):
        population_by_region.append(pop)
        win_loss_by_region.append(ratio)

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(nfl_correlation())

0.004922112149349471


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_df():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df_2018 = nhl_df.where(nhl_df['year'] == 2018).drop(['GP', 'year', 'League', 'PTS', 'PTS%', 'GF', 
                                                             'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'OL'], axis=1).dropna()
    nhl_df_2018.replace({'Atlantic Division':np.nan, 'Metropolitan Division':np.nan, 'Central Division':np.nan,
                        'Pacific Division':np.nan}, inplace=True)
    nhl_df_2018['team'].replace({'New Jersey Devils*': 'New York', 'Carolina Hurricanes': 'Raleigh',
                                'Colorado Avalanche*':'Denver', 'Vegas Golden Knights*': 'Las Vegas',
                                'Anaheim Ducks*':'Los Angeles', 'Arizona Coyotes': 'Phoenix'}, inplace=True)
    nhl_df_2018 = nhl_df_2018.dropna().reset_index().drop('index',axis=1)
    nhl_population = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    nhl_population.replace('—', np.nan, inplace=True)
    nhl_population.replace(r'^\[\w+\s\d+\]$', np.nan, regex=True,inplace=True)
    nhl_population.replace({'New York City': 'New York', 'San Francisco Bay Area': 'San Jose', 'Dallas–Fort Worth': 'Dallas',
                           'Washington, D.C.': 'Washington', 'Minneapolis–Saint Paul':"Minnesota", 
                            'Miami–Fort Lauderdale': 'Florida', 'Tampa Bay Area': 'Tampa Bay'}, inplace=True)
    nhl_population = nhl_population.dropna().reset_index().drop('index', axis=1)
    region= {}
    population= {} 
    for area_idx, area in enumerate(nhl_population['Metropolitan area'].to_list(), start=0):
        for team, team_idx in zip(nhl_df_2018['team'].to_list(), nhl_df_2018.index.to_list()):
            if re.match(area, team) != None:
                region[team_idx] = area
                population[team_idx] = nhl_population['Population (2016 est.)[8]'][area_idx]
    region_df = pd.Series(region, name='region')
    population_df = pd.Series(population, name='population')
    population_df = pd.to_numeric(population_df)
    nhl_df_2018['region'] = region_df
    nhl_df_2018['population'] = population_df
    nhl_df_2018['team'].replace({'New York':'New Jersey Devils*', 'Raleigh':'Carolina Hurricanes',
                                'Denver':'Colorado Avalanche*', 'Las Vegas': 'Vegas Golden Knights*',
                                'Los Angeles':'Anaheim Ducks*', 'Phoenix':'Arizona Coyotes'}, inplace=True)
    nhl_df_2018['W'] = nhl_df_2018['W'].astype(float)
    nhl_df_2018['L'] = nhl_df_2018['L'].astype(float)
    nhl_df_2018 = nhl_df_2018.set_index('region')
    new_york = nhl_df_2018.loc['New York']
    los_angeles = nhl_df_2018.loc['Los Angeles']
    ny_wlratio = nhl_df_2018.loc['New York'][['W','L']].sum() / len(nhl_df_2018.loc['New York'][['W','L']])
    la_wlratio = nhl_df_2018.loc['Los Angeles'][['W','L']].sum() / len(nhl_df_2018.loc['Los Angeles'][['W','L']])
    nhl_df_2018.drop(['New York', 'Los Angeles'], inplace=True)
    nhl_df_2018.drop('team', axis=1, inplace=True)
    nhl_df_2018.loc['New York'] = [ny_wlratio['W'], ny_wlratio['L'], new_york['population'].to_list()[0]]
    nhl_df_2018.loc['Los Angeles'] = [la_wlratio['W'], la_wlratio['L'], los_angeles['population'].to_list()[0]]
    nhl_df_2018['wl_ratio'] = nhl_df_2018['W'] / (nhl_df_2018['W']+nhl_df_2018['L'])
    nhl_df_2018.drop(['W','L'], axis=1, inplace=True)
    nhl_df_2018 = nhl_df_2018.reset_index()
    
    return nhl_df_2018

def nba_df():
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df.drop(['W/L%', 'PS/G', 'PA/G', 'SRS' ,'League', 'GB'], axis=1, inplace=True)
    nba_df = nba_df.where(nba_df['year'] == 2018).dropna()
    nba_df.drop('year', axis=1, inplace=True)
    
    # cities modifications and cleaning.
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.drop(['NFL','MLB','NHL'], axis=1, inplace=True)
    cities.replace([r'^\[\w+\s\d+\]$', '—'], np.nan, regex=True,inplace=True)
    cities = cities.dropna().reset_index()
    cities.rename({'Population (2016 est.)[8]':'population'}, axis=1, inplace=True)
    cities.drop('index', axis=1, inplace=True)
    cities.replace({'New York City': 'New York', 'San Francisco Bay Area': 'Golden State', 'Dallas–Fort Worth': 'Dallas',
                   'Washington, D.C.': 'Washington', 'Minneapolis–Saint Paul':'Minnesota', 'Miami–Fort Lauderdale': 'Miami',
                   'Salt Lake City': 'Utah' ,'Indianapolis': 'Indiana'}, inplace=True) 

    # create region and population columns
    population = {}
    region = {}
    for idx, city in enumerate(cities['Metropolitan area'].to_list(), start=0):
        for city2, idx2 in zip(nba_df['team'].to_list(), nba_df.index.to_list()):
            if re.match(city, city2) != None:
                population[idx2] = cities['population'][idx]
                region[idx2] = cities['Metropolitan area'][idx]
            if city2 == 'Brooklyn Nets\xa0(12)':
                population[idx2] = cities['population'][0]
                region[idx2] = cities['Metropolitan area'][0]
        
    nba_df['region'] = pd.Series(region)
    nba_df['population'] = pd.Series(population)
    
    # treat los angeles and new york entries differently.
    nba_df[['W', 'L']] = nba_df[['W', 'L']].astype(float)
    
    la_reg, la_pop = nba_df.iloc[24:26]['region'].to_list()[0], nba_df.iloc[24:26]['population'].to_list()[0]
    la_w, la_l = nba_df.iloc[24:26]['W'].sum() / len(nba_df.iloc[24:26]), nba_df.iloc[24:26]['L'].sum() / len(nba_df.iloc[24:26])
    
    ny_reg, ny_pop = nba_df.iloc[10:12]['region'].to_list()[0], nba_df.iloc[10:12]['population'].to_list()[0]
    ny_w, ny_l = nba_df.iloc[10:12]['W'].sum() / len(nba_df.iloc[10:12]), nba_df.iloc[10:12]['L'].sum() / len(nba_df.iloc[10:12])

    nba_df.drop('team', axis=1, inplace=True)
    nba_df.drop([10, 11, 24, 25], axis=0 , inplace=True)
    nba_df = nba_df.reset_index()
    nba_df.drop('index', axis=1, inplace=True)
    
    nba_df.loc[len(nba_df)] = [ny_w, ny_l, ny_reg, ny_pop]
    nba_df.loc[len(nba_df)] = [la_w, la_l, la_reg, la_pop]
    
    nba_df['wl_ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    nba_df['population'] = nba_df['population'].astype(float)
    nba_df.drop(['W','L'], axis=1, inplace=True)
    return nba_df

def sports_team_performance():
    # YOUR CODE HERE
    nhl_dataframe = nhl_df()
    nba_dataframe = nba_df()
    
    # NHL - NBA 
    nhl_nba_x = {}
    nhl_nba_y = {}
    for idx, region1 in enumerate(nhl_dataframe['region'].to_list(), start=0):
        for idx2, region2 in enumerate(nba_dataframe['region'].to_list(), start=0):
            if region1 == region2 :
                nhl_nba_x[idx] = nhl_dataframe['wl_ratio'][idx]
                nhl_nba_y[idx] = nba_dataframe['wl_ratio'][idx2]
            if region1 == 'San Jose' and region2 == 'San Antonio':
                nhl_nba_x[idx] = nhl_dataframe['wl_ratio'][idx]
                nhl_nba_y[idx] = nba_dataframe['wl_ratio'][idx2]
            if region1 == 'Florida' and region2 == 'Miami':
                nhl_nba_x[idx] = nhl_dataframe['wl_ratio'][idx]
                nhl_nba_y[idx] = nba_dataframe['wl_ratio'][idx2]
            
    nhl_nba_df = nhl_dataframe.copy()
    nhl_nba_df['wl_ratio_x'] = pd.Series(nhl_nba_x)
    nhl_nba_df['wl_ratio_y'] = pd.Series(nhl_nba_y)
    nhl_nba_df = nhl_nba_df.dropna()
    
    return stats.ttest_rel(nhl_nba_df['wl_ratio_x'].to_list(), nhl_nba_df['wl_ratio_y'].to_list())[1]

print(sports_team_performance())

0.00641220397527703
